In [1]:
import os

In [2]:
os.chdir("../")

In [3]:

%pwd

'd:\\LaboratoireDeveloppement_TI\\IA\\Machine Learning\\deeplearningproject\\Chicken-Disease-Classification-Project'

In [4]:
import os
import sys
import time
import tensorflow as tf
from pathlib import Path
from src.constants import *
from zipfile import ZipFile
import urllib.request as request
from dataclasses import dataclass
from src.exception.exception import CustomException
from src.utils.common import fn_read_yaml, fn_create_directories


@dataclass(frozen=True)
class _PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path   

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = fn_read_yaml(config_filepath)
        self.params = fn_read_yaml(params_filepath)      
        
        fn_create_directories([self.config.artifacts_root])
        
        
    def fn_get_prepare_callback_config(self) -> _PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        fn_create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = _PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
            
        return prepare_callback_config

In [6]:
class PrepareCallback:
    def __init__(self, config: _PrepareCallbacksConfig):
        self.config = config
        
        
    @property
    def fn_create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    
    @property
    def fn_create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
        
    def fn_get_tb_ckpt_callbacks(self):
        return [
            self.fn_create_tb_callbacks,
            self.fn_create_ckpt_callbacks
        ]

In [7]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.fn_get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.fn_get_tb_ckpt_callbacks()
except Exception as e:
    raise CustomException(e, sys)